# Northwind Data Visualization - Interactive Dashboard

This notebook contains interactive visualizations using Plotly, including delivery statistics and 3D analysis.

In [1]:
# Install required packages if not already installed
%pip install plotly nbformat pandas ipywidgets --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

os.makedirs("../figures", exist_ok=True)

In [3]:
# Load the denormalized data from warehouse
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

data_path = "../data/warehouse/merged_northwind.csv"
if not os.path.exists(data_path):
    print(f"File not found: {data_path}. Please run the ETL pipeline first.")
else:
    df = pd.read_csv(data_path)
    df['FullDate'] = pd.to_datetime(df['FullDate'])
    print("Data loaded successfully from merged_northwind.csv")
    print(f"Total records: {len(df)}")
    print(df.head())

Data loaded successfully from merged_northwind.csv
Total records: 48
   OrderId  EmployeeId  CustomerId           Order Date Shipped Date  \
0       30           9          27  2006-01-15 00:00:00   2006-01-22   
1       31           3           4  2006-01-20 00:00:00   2006-01-22   
2       32           4          12  2006-01-22 00:00:00   2006-01-22   
3       33           6           8  2006-01-30 00:00:00   2006-01-31   
4       34           9           4  2006-02-06 00:00:00   2006-02-07   

   Shipper ID           Ship Name     Ship Address  Ship City  \
0         2.0           Karen Toh  789 27th Street  Las Vegas   
1         1.0       Christina Lee   123 4th Street   New York   
2         2.0        John Edwards  123 12th Street  Las Vegas   
3         3.0  Elizabeth Andersen   123 8th Street   Portland   
4         3.0       Christina Lee   123 4th Street   New York   

  Ship State/Province  ...     City_x Country_x  FirstName        LastName  \
0                  NV  ...  L

## 1. Delivery Status Overview

In [4]:
if 'df' in locals():
    delivery_counts = df['DeliveredFlag'].value_counts()
    
    fig = go.Figure(data=[go.Pie(
        labels=['Delivered', 'Not Delivered'],
        values=[delivery_counts.get(1, 0), delivery_counts.get(0, 0)],
        hole=0.3,
        marker=dict(colors=['#2ecc71', '#e74c3c']),
        textinfo='label+percent+value',
        hovertemplate='<b>%{label}</b><br>Count: %{value}<br>Percentage: %{percent}<extra></extra>'
    )])
    
    fig.update_layout(
        title='Order Delivery Status',
        font=dict(size=14),
        height=500
    )
    
    try:
        fig.write_html("../figures/delivery_stats_notebook.html")
        fig.show()
    except Exception as e:
        print(f"Error displaying/saving plot: {e}")

## 2. Orders by Country (Interactive)

In [5]:
if 'df' in locals():
    country_orders = df['Country_x'].value_counts().reset_index()
    country_orders.columns = ['Country', 'OrderCount']
    
    fig = px.bar(
        country_orders,
        x='Country',
        y='OrderCount',
        title='Total Orders by Country',
        color='OrderCount',
        color_continuous_scale='Viridis',
        hover_data={'OrderCount': ':,'}
    )
    
    fig.update_layout(
        xaxis_title='Country',
        yaxis_title='Number of Orders',
        height=600
    )
    
    try:
        fig.write_html("../figures/orders_by_country_notebook.html")
        fig.show()
    except Exception as e:
        print(f"Error displaying/saving plot: {e}")

## 3. Orders by Employee (Interactive)

In [6]:
if 'df' in locals():
    df['EmployeeName'] = df['FirstName'] + ' ' + df['LastName']
    employee_orders = df['EmployeeName'].value_counts().reset_index()
    employee_orders.columns = ['EmployeeName', 'OrderCount']
    
    fig = px.bar(
        employee_orders,
        y='EmployeeName',
        x='OrderCount',
        orientation='h',
        title='Orders by Employee',
        color='OrderCount',
        color_continuous_scale='Plasma'
    )
    
    fig.update_layout(
        xaxis_title='Number of Orders',
        yaxis_title='Employee',
        height=600
    )
    
    try:
        fig.write_html("../figures/orders_by_employee_notebook.html")
        fig.show()
    except Exception as e:
        print(f"Error displaying/saving plot: {e}")

## 4. Monthly Orders Trend (Interactive)

In [7]:
if 'df' in locals():
    df['YearMonth'] = df['FullDate'].dt.to_period('M').astype(str)
    monthly_orders = df.groupby('YearMonth').size().reset_index(name='OrderCount')
    
    fig = px.line(
        monthly_orders,
        x='YearMonth',
        y='OrderCount',
        title='Monthly Orders Trend',
        markers=True
    )
    
    fig.update_traces(
        line=dict(color='#3498db', width=3),
        marker=dict(size=8)
    )
    
    fig.update_layout(
        xaxis_title='Month',
        yaxis_title='Number of Orders',
        height=600,
        hovermode='x unified'
    )
    
    try:
        fig.write_html("../figures/monthly_trend_notebook.html")
        fig.show()
    except Exception as e:
        print(f"Error displaying/saving plot: {e}")

## 5. 3D Interactive Visualization: Orders by Month and Country

In [8]:
# --- MOCK DATA GENERATION (1996-2005) ---
# The source data only contains 2006. We generate mock history for visualization purposes.
import numpy as np

if 'df' in locals():
    print("Generating mock historical data (1996-2005)...")
    

    customers = df['CompanyName'].unique()
    employees = df['LastName'].unique()
    
    mock_rows = []

    for _ in range(800):

        year = np.random.randint(1996, 2006)

        month = np.random.randint(1, 13)
        day = np.random.randint(1, 28)
        mock_date = pd.Timestamp(year=year, month=month, day=day)
        
        row = {
            'CompanyName': np.random.choice(customers),
            'LastName': np.random.choice(employees),
            'FullDate': mock_date,
            'Year': year,
            'OrderCount': 1, 
            'DeliveredFlag': np.random.choice([0, 1]),
            'Country_x': 'USA'
        }
        mock_rows.append(row)
    
    mock_df = pd.DataFrame(mock_rows)

    if 'FullDate' in df.columns:
        df['FullDate'] = pd.to_datetime(df['FullDate'])
        df['Year'] = df['FullDate'].dt.year

    df = pd.concat([df, mock_df], ignore_index=True)
    
   
    df['FullDate'] = pd.to_datetime(df['FullDate'])
    df['MonthName'] = df['FullDate'].dt.month_name()
    df['Year'] = df['FullDate'].dt.year

    print(f"Added {len(mock_df)} mock records. Total records: {len(df)}")
    print(f"Years present: {sorted(df['Year'].unique())}")

Generating mock historical data (1996-2005)...
Added 800 mock records. Total records: 848
Years present: [np.int32(1996), np.int32(1997), np.int32(1998), np.int32(1999), np.int32(2000), np.int32(2001), np.int32(2002), np.int32(2003), np.int32(2004), np.int32(2005), np.int32(2006)]


In [9]:

import plotly.express as px
import pandas as pd
import itertools
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

if 'df' in locals():

    if 'FullDate' in df.columns:
        df['FullDate'] = pd.to_datetime(df['FullDate'])
        df['Year'] = df['FullDate'].dt.year
        df['MonthName'] = df['FullDate'].dt.month_name()
        df['DateStr'] = df['FullDate'].dt.strftime('%Y-%m-%d')

    months_order = ['January', 'February', 'March', 'April', 'May', 'June', 
                   'July', 'August', 'September', 'October', 'November', 'December']
    df['MonthName'] = pd.Categorical(df['MonthName'], categories=months_order, ordered=True)
    

    df_monthly = df.groupby(['Year', 'Country_x', 'MonthName', 'DeliveredFlag'], observed=False).size().reset_index(name='OrderCount')
    df_monthly['DeliveryStatus'] = df_monthly['DeliveredFlag'].map({1: 'Delivered', 0: 'Not Delivered'})
    df_monthly = df_monthly.sort_values(['Year', 'MonthName'])
 
    df_olap = df.groupby(['Year', 'CompanyName', 'LastName', 'FullDate', 'DeliveredFlag'], observed=False).size().reset_index(name='OrderCount')
    df_olap['DeliveryStatus'] = df_olap['DeliveredFlag'].map({1: 'Delivered', 0: 'Not Delivered'})
    df_olap = df_olap.sort_values(['Year', 'FullDate'])

    min_date = df['FullDate'].min()
    max_date = df['FullDate'].max()
    all_dates = pd.date_range(start=min_date, end=max_date, freq='D')
    countries = df['Country_x'].unique()
    
    full_grid = pd.DataFrame(list(itertools.product(all_dates, countries)), columns=['FullDate', 'Country_x'])
    full_grid['Year'] = full_grid['FullDate'].dt.year
    full_grid['DateStr'] = full_grid['FullDate'].dt.strftime('%Y-%m-%d')
    
    daily_agg = df.groupby(['Year', 'Country_x', 'FullDate', 'DateStr', 'DeliveredFlag'], observed=False).size().reset_index(name='OrderCount')
    existing_combinations = daily_agg[['Country_x', 'FullDate']].drop_duplicates()
    
    merged = full_grid.merge(existing_combinations, on=['Country_x', 'FullDate'], how='left', indicator=True)
    df_missing = merged[merged['_merge'] == 'left_only'].drop(columns=['_merge']).copy()
    df_missing['OrderCount'] = 0
    df_missing['DeliveryStatus'] = 'No Data'
    df_missing['DeliveredFlag'] = -1

    
    def plot_3d(view_type, color_by, show_grid, symbol_coding, year, country):
        fig = None

        layout_args = dict(
            height=700
        )
        
        if view_type == 'Monthly Trends':

            data = df_monthly.copy()

            if year != 'All':
                data = data[data['Year'] == int(year)]
            if country != 'All':
                 data = data[data['Country_x'] == country]

            c_col = 'DeliveryStatus' if color_by == 'Status' else 'Country_x'
            
            if data.empty:
                print('No data for selected filters.')
                return

            fig = px.scatter_3d(
                data,
                x='Country_x',
                y='MonthName',
                z='OrderCount',
                color=c_col,
                symbol='DeliveryStatus' if symbol_coding else None,
                size='OrderCount',
                size_max=30,
                animation_frame='Year' if year == 'All' else None,
                animation_group='Country_x',
                title=f'Monthly Order Trends ({year})',
                labels={'Country_x': 'Country', 'MonthName': 'Month', 'OrderCount': 'Orders'},
                **layout_args
            )
            fig.update_layout(scene=dict(xaxis_title='Country', yaxis_title='Month', zaxis_title='Count'), margin=dict(r=0, l=0, b=0, t=40))

        elif view_type == 'OLAP Cube':
            data = df_olap.copy()
     
            if year != 'All':
                data = data[data['Year'] == int(year)]

            if data.empty:
                print('No data for selected filters.')
                return

            c_col = 'DeliveryStatus' if color_by == 'Status' else 'CompanyName'

            fig = px.scatter_3d(
                data,
                x='CompanyName',
                y='LastName',
                z='FullDate',
                color=c_col,
                symbol='DeliveryStatus' if symbol_coding else None,
                size='OrderCount',
                size_max=20,
                animation_frame='Year' if year == 'All' else None,
                animation_group='CompanyName',
                title=f'OLAP: Client vs Employee vs Date ({year})',
                labels={'CompanyName': 'Client', 'LastName': 'Employee', 'FullDate': 'Date'},
                **layout_args
            )
            fig.update_layout(scene=dict(xaxis_title='Client', yaxis_title='Employee', zaxis_title='Date'), margin=dict(r=0, l=0, b=0, t=40))

        elif view_type == 'Daily Status':
            data = daily_agg.copy()
            
            if year != 'All':
                data = data[data['Year'] == int(year)]
            if country != 'All':
                 data = data[data['Country_x'] == country]
                 
            data['VisualSize'] = data['OrderCount']
            
            if show_grid:
                grid_data = df_missing.copy()
                grid_data['VisualSize'] = 1 
                data = pd.concat([data, grid_data], ignore_index=True)
                
            data = data.sort_values(['FullDate', 'Country_x'])
            
          
            color_map = {}
            if color_by == 'Status':
                c_col = 'DeliveryStatus'
                color_map = {'Delivered': '#00CC96', 'Not Delivered': '#EF553B', 'No Data': 'grey'}
            else:
                 c_col = 'Country_x'
                 
                 pass

            fig = px.scatter_3d(
                data,
                x='Country_x',
                y='FullDate',
                z='OrderCount',
                color=c_col,
                symbol='DeliveryStatus' if symbol_coding else None,
                size='VisualSize',
                size_max=30,
                title='Daily Order Status',
                labels={'Country_x': 'Country', 'FullDate': 'Date'},
                color_discrete_map=color_map if color_map else None,
                **layout_args
            )
            fig.update_layout(scene=dict(xaxis_title='Country', yaxis_title='Date', zaxis_title='Count'), margin=dict(r=0, l=0, b=0, t=40))
        
        if fig:
            fig.show()

  
    unique_years = sorted(df['Year'].unique())
    unique_countries = sorted(df['Country_x'].unique())
    
    w_view = widgets.Dropdown(options=['Monthly Trends', 'Daily Status', 'OLAP Cube'], value='Monthly Trends', description='View Type')
    w_color = widgets.Dropdown(options=['Status', 'Entity'], value='Status', description='Color By')
    w_grid = widgets.Checkbox(value=False, description='Show Empty Grid (Daily)')
    w_symbol = widgets.Checkbox(value=True, description='Use Shapes')
    w_year = widgets.Dropdown(options=['All'] + [str(y) for y in unique_years], value='All', description='Year')
    w_country = widgets.Dropdown(options=['All'] + list(unique_countries), value='All', description='Country')
    
    # Render
    print("Select options to customize the 3D visualization:")
    interact(plot_3d, view_type=w_view, color_by=w_color, show_grid=w_grid, symbol_coding=w_symbol, year=w_year, country=w_country);

Select options to customize the 3D visualization:


interactive(children=(Dropdown(description='View Type', options=('Monthly Trends', 'Daily Status', 'OLAP Cube'…

## 6. Delivery Status by Country

In [10]:
if 'df' in locals():
    delivery_by_country = df.groupby(['Country_x', 'DeliveredFlag']).size().reset_index(name='Count')
    delivery_by_country['Status'] = delivery_by_country['DeliveredFlag'].map({1: 'Delivered', 0: 'Not Delivered'})
    
    fig = px.bar(
        delivery_by_country,
        x='Country_x',
        y='Count',
        color='Status',
        title='Delivery Status by Country',
        barmode='stack',
        color_discrete_map={'Delivered': '#2ecc71', 'Not Delivered': '#e74c3c'}
    )
    
    fig.update_layout(
        xaxis_title='Country',
        yaxis_title='Number of Orders',
        height=600
    )
    
    try:
        fig.write_html("../figures/delivery_by_country_notebook.html")
        fig.show()
    except Exception as e:
        print(f"Error displaying/saving plot: {e}")

## 7. Comprehensive Dashboard

In [11]:
if 'df' in locals():
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Delivery Status', 'Orders by Country', 'Monthly Trend', 'Top Employees'),
        specs=[[{'type': 'pie'}, {'type': 'bar'}],
               [{'type': 'scatter'}, {'type': 'bar'}]]
    )
    
    
    delivery_counts = df['DeliveredFlag'].value_counts()
    fig.add_trace(
        go.Pie(labels=['Delivered', 'Not Delivered'],
               values=[delivery_counts.get(1, 0), delivery_counts.get(0, 0)],
               marker=dict(colors=['#2ecc71', '#e74c3c'])),
        row=1, col=1
    )
    

    country_orders = df['Country_x'].value_counts().head(5).reset_index()
    country_orders.columns = ['Country', 'Count']
    fig.add_trace(
        go.Bar(x=country_orders['Country'], y=country_orders['Count'],
               marker=dict(color='#3498db')),
        row=1, col=2
    )

    df['YearMonth'] = df['FullDate'].dt.to_period('M').astype(str)
    monthly = df.groupby('YearMonth').size().reset_index(name='Count')
    fig.add_trace(
        go.Scatter(x=monthly['YearMonth'], y=monthly['Count'],
                   mode='lines+markers', line=dict(color='#9b59b6')),
        row=2, col=1
    )

    df['EmployeeName'] = df['FirstName'] + ' ' + df['LastName']
    employee_orders = df['EmployeeName'].value_counts().head(5).reset_index()
    employee_orders.columns = ['Employee', 'Count']
    fig.add_trace(
        go.Bar(y=employee_orders['Employee'], x=employee_orders['Count'],
               orientation='h', marker=dict(color='#e67e22')),
        row=2, col=2
    )
    
    fig.update_layout(
        title_text='Northwind Orders Dashboard',
        showlegend=False,
        height=900
    )
    
    try:
        fig.write_html("../figures/dashboard_notebook.html")
        fig.show()
    except Exception as e:
        print(f"Error displaying/saving plot: {e}")

## 4. Detailed Delivery Analysis
Analysis of delivery efficiency by Employee and Country.

In [12]:

if 'df' in locals():

    emp_delivery = df.groupby(['LastName', 'DeliveredFlag']).size().reset_index(name='Count')

    emp_delivery['Status'] = emp_delivery['DeliveredFlag'].map({1: 'Delivered', 0: 'Not Delivered'})
    
    fig_emp = px.bar(emp_delivery, x='LastName', y='Count', color='Status', 
                     title='Delivery Status by Employee', 
                     labels={'LastName': 'Employee', 'Count': 'Number of Orders'},
                     barmode='stack')
    
    country_delivery = df.groupby(['Country_x', 'DeliveredFlag']).size().reset_index(name='Count')
    country_delivery['Status'] = country_delivery['DeliveredFlag'].map({1: 'Delivered', 0: 'Not Delivered'})
    
    fig_country = px.bar(country_delivery, x='Country_x', y='Count', color='Status', 
                         title='Delivery Status by Country', 
                         labels={'Country_x': 'Country', 'Count': 'Number of Orders'},
                         barmode='stack')
    
    fig_emp.update_layout(height=500)
    fig_country.update_layout(height=500)
    
    try:
        fig_emp.show()
        fig_country.show()
        fig_emp.write_html("../figures/delivery_by_employee.html")
        fig_country.write_html("../figures/delivery_by_country.html")
    except Exception as e:
        print(f"Error displaying/saving plot: {e}")

In [13]:
# 7. 3D View of Individual Orders (Detail View)
# Now using the comprehensive merged dataset
import plotly.express as px
import os

try:
    if 'df' in locals():
        # Using columns directly from the merged dataset
        plot_df = df.copy()
        plot_df['DeliveryStatus'] = plot_df['DeliveredFlag'].map({1: 'Delivered', 0: 'Not Delivered'})
        plot_df['Employee'] = plot_df['FirstName'] + ' ' + plot_df['LastName']
        
        # Use available columns from Access
        country_col = 'Ship Country/Region' if 'Ship Country/Region' in plot_df.columns else 'Country_x'
        fee_col = 'Shipping Fee' if 'Shipping Fee' in plot_df.columns else 'OrderId'
        
        # Check if Ship Name exists
        hover_cols = ['OrderId', 'Ship Name', 'FirstName', 'LastName', 'Payment Type', 'Shipping Fee']
        hover_cols = [c for c in hover_cols if c in plot_df.columns]
        
        fig_detail = px.scatter_3d(
            plot_df,
            x=country_col,
            y='FullDate',
            z=fee_col,
            color='DeliveryStatus',
            symbol='Payment Type' if 'Payment Type' in plot_df.columns else None,
            hover_data=hover_cols,
            title='3D Individual Orders: Fee vs Date vs Country',
            labels={country_col: 'Country', fee_col: 'Shipping Fee ($)', 'FullDate': 'Date'},
            height=800,
            color_discrete_map={'Delivered': '#00CC96', 'Not Delivered': '#EF553B', 'Unknown': 'grey'}
        )
        
        fig_detail.update_layout(
            scene=dict(
                xaxis_title='Country',
                yaxis_title='Date',
                zaxis_title='Shipping Fee ($)'
            ),
            margin=dict(r=0, l=0, b=0, t=40),
            template="plotly_dark"
        )
        
        output_file = "../figures/individual_orders_3d.html"
        fig_detail.write_html(output_file)
        print(f"Individual Orders 3D plot saved to {output_file}")
        fig_detail.show()
    else:
        print("DataFrame 'df' not found.")
        
except Exception as e:
    print(f"Error creating individual orders plot: {e}")

Individual Orders 3D plot saved to ../figures/individual_orders_3d.html


## 6. Employee Order Selection (Interactive Explorer)

Select an employee from the dropdown below to view their specific order trends.

In [14]:
import ipywidgets as widgets
from IPython.display import display

# Ensure df has EmployeeName
if 'df' in locals():
    if 'FirstName' in df.columns and 'LastName' in df.columns:
        df['EmployeeName'] = df['FirstName'].astype(str) + ' ' + df['LastName'].astype(str)
    
    employees = sorted(df['EmployeeName'].unique())

    # Create dropdown
    dropdown = widgets.Dropdown(
        options=employees,
        value=employees[0] if employees else None,
        description='Employee:',
        style={'description_width': 'initial'}
    )

    # Output area
    out = widgets.Output()

    def update_chart(change):
        emp_name = change['new']
        with out:
            out.clear_output()
            emp_df = df[df['EmployeeName'] == emp_name].copy()
            if not emp_df.empty:
                emp_df['YearMonth'] = emp_df['FullDate'].dt.to_period('M').astype(str)
                monthly = emp_df.groupby('YearMonth').size().reset_index(name='OrderCount')
                
                fig = px.bar(
                    monthly,
                    x='YearMonth',
                    y='OrderCount',
                    title=f"Order Trend for {emp_name}",
                    color='OrderCount',
                    color_continuous_scale='Viridis',
                    template="plotly_dark"
                )
                fig.update_layout(height=500)
                fig.show()
            else:
                print(f"No data found for {emp_name}")

    dropdown.observe(update_chart, names='value')

    print("Choose an employee to explore their data:")
    display(dropdown)
    display(out)

    # Trigger first update
    if employees:
        update_chart({'new': employees[0]})
else:
    print("DataFrame 'df' not found. Please run the data loading cells above.")

Choose an employee to explore their data:


Dropdown(description='Employee:', options=('Andrew Cencini', 'Anne Hellung-Larsen', 'Jan Kotas', 'Laura Giussa…

Output()